<a href="https://colab.research.google.com/github/tlsgptj/2024-Samsung-AI-Challenge-Black-box-Optimization/blob/main/2024_Samsung_AI_Challenge_Black_box_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#사용가능한 알고리즘
# 1. Xgboost
# 2. 랜덤 포레스트 회귀 (Random Forest Regressor)
# 3. 라이트GBM (LightGBM)

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
X_train = train_df.drop(columns=['ID', 'y'])  # 'ID'와 'y'를 제외한 특징 사용
y_train = train_df['y']

In [ ]:
X_test = test_df.drop(columns=['ID'])  # 'ID'를 제외한 특징 사용
test_ids = test_df['ID']  # 제출을 위한 ID

In [ ]:
!pip install scikit-optimize

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from skopt import BayesSearchCV

In [ ]:
rf = RandomForestRegressor(random_state=42)

In [ ]:
param_space = {
    'n_estimators': (10, 100),
    'max_depth': (1, 20),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 10)
}

In [ ]:
opt = BayesSearchCV(estimator=rf, search_spaces=param_space, n_iter=32, cv=3, n_jobs=-1, random_state=42)
opt.fit(X_train, y_train)

In [ ]:
print("Best hyperparameters:", opt.best_params_)

# 최적 모델로 테스트 데이터 예측
y_test_pred = opt.predict(X_test)

Best hyperparameters: OrderedDict([('max_depth', 7), ('min_samples_leaf', 10), ('min_samples_split', 2), ('n_estimators', 100)])

In [ ]:
# 성능 평가 (학습 데이터에서 성능 검증)
# 여기서는 train 데이터의 20%를 검증용으로 다시 나누어 성능 평가
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
opt.fit(X_train_part, y_train_part)
y_valid_pred = opt.predict(X_valid)

In [ ]:
mse = mean_squared_error(y_valid, y_valid_pred)
r2 = r2_score(y_valid, y_valid_pred)

In [ ]:
print(f"Validation Mean Squared Error: {mse:.4f}")
print(f"Validation R-squared: {r2:.4f}")

In [ ]:
threshold = np.percentile(y_test_pred, 67)
top_33_percent_mask = y_test_pred >= threshold

In [ ]:
# 제출 파일 생성
submission_df = pd.read_csv('sample_submission.csv')
submission_df['y'] = y_test_pred
submission_df.to_csv('updated_submission.csv', index=False)

print(f"Top 33% threshold: {threshold:.4f}")
print(f"Number of samples in top 33%: {sum(top_33_percent_mask)}")
print("Submission file 'updated_submission.csv' created successfully.")

Top 33% threshold: 84.5243
Number of samples in top 33%: 1646
Submission file 'submission.csv' created successfully.

In [ ]:
!pip install scikit-optimize

In [ ]:
#XgBoost
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from skopt import BayesSearchCV
from xgboost import XGBRegressor
import numpy as np

In [ ]:
# 데이터 로드
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
X_train = train_df.drop(columns=['ID', 'y'])  # 'ID'와 'y'를 제외한 특징 사용
y_train = train_df['y']
X_test = test_df.drop(columns=['ID'])  # 'ID'를 제외한 특징 사용
test_ids = test_df['ID']  # 제출을 위한 ID

In [ ]:
xgb_model = XGBRegressor(random_state=42, objective='reg:squarederror')

In [ ]:
param_space_xgb = {
    'n_estimators': (10, 100),
    'max_depth': (1, 20),
    'learning_rate': (0.01, 0.3, 'log-uniform'),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'min_child_weight': (1, 10)
}

In [ ]:
opt_xgb = BayesSearchCV(estimator=xgb_model, search_spaces=param_space_xgb, n_iter=32, cv=3, n_jobs=-1, random_state=42)

In [ ]:
opt_xgb.fit(X_train, y_train)
print("Best hyperparameters for XGBoost:", opt_xgb.best_params_)

Best hyperparameters for XGBoost: OrderedDict([('colsample_bytree', 0.7224162561505759), ('learning_rate', 0.22754356809600707), ('max_depth', 3), ('min_child_weight', 5), ('n_estimators', 27), ('subsample', 0.7268326719031495)])

In [ ]:
y_test_pred_xgb = opt_xgb.predict(X_test)

In [ ]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
opt_xgb.fit(X_train_part, y_train_part)
y_valid_pred_xgb = opt_xgb.predict(X_valid)
mse_xgb = mean_squared_error(y_valid, y_valid_pred_xgb)
r2_xgb = r2_score(y_valid, y_valid_pred_xgb)
print(f"Validation Mean Squared Error (XGBoost): {mse_xgb:.4f}")
print(f"Validation R-squared (XGBoost): {r2_xgb:.4f}")

Validation Mean Squared Error (XGBoost): 2.5717
Validation R-squared (XGBoost): 0.6407

In [ ]:
threshold_xgb = np.percentile(y_test_pred_xgb, 67)
top_33_percent_mask_xgb = y_test_pred_xgb >= threshold_xgb

In [ ]:
submission_df_xgb = pd.read_csv('sample_submission.csv')
submission_df_xgb['y'] = y_test_pred_xgb
submission_df_xgb.to_csv('xgb_updated_submission.csv', index=False)

In [ ]:
print(f"Top 33% threshold (XGBoost): {threshold_xgb:.4f}")
print(f"Number of samples in top 33% (XGBoost): {sum(top_33_percent_mask_xgb)}")
print("Submission file 'xgb_updated_submission.csv' created successfully.")

Top 33% threshold (XGBoost): 84.5228
Number of samples in top 33% (XGBoost): 1646
Submission file 'xgb_updated_submission.csv' created successfully.

In [ ]:
!pip install CatBoost

In [ ]:
#Cat Boost
from catboost import CatBoostRegressor

In [ ]:
cat_model = CatBoostRegressor(random_state=42, verbose=0)

In [ ]:
# 하이퍼파라미터 검색 공간 정의
param_space_cat = {
    'iterations': (100, 1000),
    'depth': (3, 12),
    'learning_rate': (0.01, 0.3, 'log-uniform'),
    'l2_leaf_reg': (1, 10),
    'bagging_temperature': (0, 1.0),
}

In [ ]:
opt_cat = BayesSearchCV(estimator=cat_model, search_spaces=param_space_cat, n_iter=50, cv=3, n_jobs=-1, random_state=42)

In [ ]:
opt_cat.fit(X_train, y_train)
print("Best hyperparameters for CatBoost:", opt_cat.best_params_)

Best hyperparameters for CatBoost: OrderedDict([('bagging_temperature', 0.9213569982250611), ('depth', 12), ('iterations', 594), ('l2_leaf_reg', 9), ('learning_rate', 0.01800427693994094)])

In [ ]:
y_test_pred_cat = opt_cat.predict(X_test)

In [ ]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
opt_cat.fit(X_train_part, y_train_part)
y_valid_pred_cat = opt_cat.predict(X_valid)
mse_cat = mean_squared_error(y_valid, y_valid_pred_cat)
r2_cat = r2_score(y_valid, y_valid_pred_cat)
print(f"Validation Mean Squared Error (CatBoost): {mse_cat:.4f}")
print(f"Validation R-squared (CatBoost): {r2_cat:.4f}")

In [ ]:
threshold_cat = np.percentile(y_test_pred_cat, 67)
top_33_percent_mask_cat = y_test_pred_cat >= threshold_cat

In [ ]:
submission_df_cat = pd.read_csv('sample_submission.csv')
submission_df_cat['y'] = y_test_pred_cat
submission_df_cat.to_csv('cat_updated_submission.csv', index=False)

print(f"Top 33% threshold (CatBoost): {threshold_cat:.4f}")
print(f"Number of samples in top 33% (CatBoost): {sum(top_33_percent_mask_cat)}")
print("Submission file 'cat_updated_submission.csv' created successfully.")

In [ ]:
#LightGBM 모델
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from lightgbm import LGBMRegressor
from skopt import BayesSearchCV

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
X_train = train_df.drop(columns=['ID', 'y'])  # 'ID'와 'y'를 제외한 특징 사용
y_train = train_df['y']
X_test = test_df.drop(columns=['ID'])  # 'ID'를 제외한 특징 사용
test_ids = test_df['ID']  # 제출을 위한 ID

In [ ]:
lgbm_model = LGBMRegressor(random_state=42)

In [ ]:
param_space_lgbm = {
    'n_estimators': (100, 1000),
    'max_depth': (3, 15),
    'learning_rate': (0.01, 0.3, 'log-uniform'),
    'num_leaves': (20, 50),
    'min_child_samples': (5, 30),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0)
}

In [ ]:
opt_lgbm = BayesSearchCV(estimator=lgbm_model, search_spaces=param_space_lgbm, n_iter=50, cv=3, n_jobs=-1, random_state=42)

In [ ]:
opt_lgbm.fit(X_train, y_train)
print("Best hyperparameters for LightGBM:", opt_lgbm.best_params_)

In [ ]:
y_test_pred_lgbm = opt_lgbm.predict(X_test)

In [ ]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
opt_lgbm.fit(X_train_part, y_train_part)
y_valid_pred_lgbm = opt_lgbm.predict(X_valid)
mse_lgbm = mean_squared_error(y_valid, y_valid_pred_lgbm)
r2_lgbm = r2_score(y_valid, y_valid_pred_lgbm)
print(f"Validation Mean Squared Error (LightGBM): {mse_lgbm:.4f}")
print(f"Validation R-squared (LightGBM): {r2_lgbm:.4f}")

In [ ]:
threshold_lgbm = np.percentile(y_test_pred_lgbm, 67)
top_33_percent_mask_lgbm = y_test_pred_lgbm >= threshold_lgbm

In [ ]:
submission_df_lgbm = pd.read_csv('sample_submission.csv')
submission_df_lgbm['y'] = y_test_pred_lgbm
submission_df_lgbm.to_csv('lgbm_updated_submission.csv', index=False)

print(f"Top 33% threshold (LightGBM): {threshold_lgbm:.4f}")
print(f"Number of samples in top 33% (LightGBM): {sum(top_33_percent_mask_lgbm)}")
print("Submission file 'lgbm_updated_submission.csv' created successfully.")

In [ ]:
!pip install scikit-optimize

In [ ]:
#Feature engneering
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from skopt import BayesSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
print(train_df.head())

In [ ]:
print("Columns in train_df:", train_df.columns)

In [ ]:
# 수치형과 범주형 특성 선택
numerical_features = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = train_df.select_dtypes(include=['object']).columns.tolist()

# 'ID'와 'y'는 제거
numerical_features = [col for col in numerical_features if col not in ['ID', 'y']]
categorical_features = [col for col in categorical_features if col != 'ID']

print("Numerical features:", numerical_features)
print("Categorical features:", categorical_features)

# 결측값 대체 전략 설정
numerical_transformer = SimpleImputer(strategy='mean')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
# 컬럼 변환기를 통해 수치형과 범주형에 다른 전처리 적용
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'  # 나머지 컬럼은 그대로 유지
)

In [ ]:

# 전처리 및 모델을 포함하는 파이프라인 구성
model = XGBRegressor(random_state=42, objective='reg:squarederror')

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),  # 스케일링 추가
    ('model', model)
])

In [ ]:
# 하이퍼파라미터 검색 공간 정의
param_space_xgb = {
    'model__n_estimators': (100, 1000),
    'model__max_depth': (3, 15),
    'model__learning_rate': (0.01, 0.3, 'log-uniform'),
    'model__subsample': (0.5, 1.0),
    'model__colsample_bytree': (0.5, 1.0),
    'model__min_child_weight': (1, 10)
}

In [ ]:
opt_xgb = BayesSearchCV(estimator=pipeline, search_spaces=param_space_xgb, n_iter=50, cv=3, n_jobs=-1, random_state=42)

In [ ]:
# 학습 데이터 준비
X_train = train_df.drop(columns=['ID', 'y'])
y_train = train_df['y']

In [ ]:
# y_train의 결측값 처리
X_train = X_train[~y_train.isnull()]
y_train = y_train.dropna()

In [ ]:
# 하이퍼파라미터 검색 공간 정의
param_space_xgb = {
    'model__n_estimators': (100, 1000),
    'model__max_depth': (3, 15),
    'model__learning_rate': (0.01, 0.3, 'log-uniform'),
    'model__subsample': (0.5, 1.0),
    'model__colsample_bytree': (0.5, 1.0),
    'model__min_child_weight': (1, 10)
}

In [ ]:
# Bayesian Optimization 설정
opt_xgb = BayesSearchCV(estimator=pipeline, search_spaces=param_space_xgb, n_iter=50, cv=3, n_jobs=-1, random_state=42)

In [ ]:
# 학습 데이터 준비
X_train = train_df.drop(columns=['ID', 'y'], errors='ignore')
y_train = train_df['y']

In [ ]:
# y_train의 결측값 처리
X_train = X_train[~y_train.isnull()]
y_train = y_train.dropna()

In [ ]:
opt_xgb.fit(X_train, y_train)
print("Best hyperparameters for XGBoost:", opt_xgb.best_params_)

Best hyperparameters for XGBoost: OrderedDict([('model__colsample_bytree', 0.5), ('model__learning_rate', 0.013066302749068358), ('model__max_depth', 3), ('model__min_child_weight', 1), ('model__n_estimators', 1000), ('model__subsample', 0.8081268064180479)])

In [ ]:
# 테스트 데이터 예측
X_test = test_df.drop(columns=['ID'])
y_test_pred_xgb = opt_xgb.predict(X_test)

In [ ]:
# 성능 평가 (학습 데이터에서 성능 검증)
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
opt_xgb.fit(X_train_part, y_train_part)
y_valid_pred_xgb = opt_xgb.predict(X_valid)
mse_xgb = mean_squared_error(y_valid, y_valid_pred_xgb)
r2_xgb = r2_score(y_valid, y_valid_pred_xgb)
print(f"Validation Mean Squared Error (XGBoost): {mse_xgb:.4f}")
print(f"Validation R-squared (XGBoost): {r2_xgb:.4f}")

In [ ]:
# 상위 33% 임계값 계산
threshold_xgb = np.percentile(y_test_pred_xgb, 67)
top_33_percent_mask_xgb = y_test_pred_xgb >= threshold_xgb

In [ ]:
# 제출 파일 생성
submission_df_xgb = pd.read_csv('sample_submission.csv')
submission_df_xgb['y'] = y_test_pred_xgb
submission_df_xgb.to_csv('xgb_feature_submission.csv', index=False)

print(f"Top 33% threshold (XGBoost): {threshold_xgb:.4f}")
print(f"Number of samples in top 33% (XGBoost): {sum(top_33_percent_mask_xgb)}")
print("Submission file 'xgb_updated_submission.csv' created successfully.")

Top 33% threshold (XGBoost): 84.4860
Number of samples in top 33% (XGBoost): 1646
Submission file 'xgb_updated_submission.csv' created successfully.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from lightgbm import LGBMRegressor
from skopt import BayesSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 데이터 로드
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# 데이터 확인
print("Train DataFrame columns:", train_df.columns)

# Feature Engineering

# 수치형과 범주형 특성 선택
numerical_features = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = train_df.select_dtypes(include=['object']).columns.tolist()

# 'ID'와 'y'는 제거
numerical_features = [col for col in numerical_features if col not in ['ID', 'y']]
categorical_features = [col for col in categorical_features if col != 'ID']

print("Numerical features:", numerical_features)
print("Categorical features:", categorical_features)

# 결측값 대체 전략 설정
numerical_transformer = SimpleImputer(strategy='mean')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 컬럼 변환기를 통해 수치형과 범주형에 다른 전처리 적용
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'  # 나머지 컬럼은 그대로 유지
)

# 전처리 및 모델을 포함하는 파이프라인 구성
model = LGBMRegressor(random_state=42, objective='regression', early_stopping_rounds=10)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),  # 스케일링 추가
    ('model', model)
])

# 하이퍼파라미터 검색 공간 정의
param_space_lgbm = {
    'model__n_estimators': (50, 2000),
    'model__max_depth': (1, 20),
    'model__learning_rate': (0.005, 0.3, 'log-uniform'),
    'model__subsample': (0.4, 1.0),
    'model__colsample_bytree': (0.4, 1.0),
    'model__min_child_samples': (1, 50),
    'model__reg_alpha': (0.0, 1.0),  # L1 regularization term
    'model__reg_lambda': (0.0, 1.0)  # L2 regularization term
}

# Bayesian Optimization 설정
opt_lgbm = BayesSearchCV(
    estimator=pipeline,
    search_spaces=param_space_lgbm,
    n_iter=100,
    cv=KFold(n_splits=5, shuffle=True, random_state=42),  # KFold 사용
    n_jobs=-1,
    random_state=42
)

# 학습 데이터 준비
X_train = train_df.drop(columns=['ID', 'y'])
y_train = train_df['y']

# 결측값 확인 및 처리
print("Missing values in X_train before:", X_train.isnull().sum().sum())
print("Missing values in y_train before:", y_train.isnull().sum())

# X_train의 결측값은 SimpleImputer로 처리되므로 따로 제거하지 않음
y_train = y_train.fillna(y_train.median())  # y_train의 결측값은 중앙값으로 대체

# 결측값 확인 후
print("Missing values in X_train after:", X_train.isnull().sum().sum())
print("Missing values in y_train after:", y_train.isnull().sum())

# 모델 학습 및 최적의 하이퍼파라미터 찾기
opt_lgbm.fit(X_train, y_train)
print("Best hyperparameters for LightGBM:", opt_lgbm.best_params_)

# 특성 중요도 확인
best_model = opt_lgbm.best_estimator_.named_steps['model']
feature_importances = best_model.feature_importances_

# 중요도에 따라 특성 정렬
feature_importances_df = pd.DataFrame({
    'feature': numerical_features + list(best_model.booster_.feature_name()),  # one-hot encoding 후 실제 특성 이름 사용
    'importance': feature_importances
}).sort_values(by='importance', ascending=False)

print(feature_importances_df)

# 테스트 데이터 예측
X_test = test_df.drop(columns=['ID'])
y_test_pred_lgbm = opt_lgbm.predict(X_test)

# 성능 평가 (학습 데이터에서 성능 검증)
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
opt_lgbm.fit(X_train_part, y_train_part)
y_valid_pred_lgbm = opt_lgbm.predict(X_valid)
mse_lgbm = mean_squared_error(y_valid, y_valid_pred_lgbm)
r2_lgbm = r2_score(y_valid, y_valid_pred_lgbm)
print(f"Validation Mean Squared Error (LightGBM): {mse_lgbm:.4f}")
print(f"Validation R-squared (LightGBM): {r2_lgbm:.4f}")

# 상위 33% 임계값 계산
threshold_lgbm = np.percentile(y_test_pred_lgbm, 67)
top_33_percent_mask_lgbm = y_test_pred_lgbm >= threshold_lgbm

# 제출 파일 생성
submission_df_lgbm = pd.read_csv('sample_submission.csv')
submission_df_lgbm['y'] = y_test_pred_lgbm
submission_df_lgbm.to_csv('lgbm_updated_submission.csv', index=False)

print(f"Top 33% threshold (LightGBM): {threshold_lgbm:.4f}")
print(f"Number of samples in top 33% (LightGBM): {sum(top_33_percent_mask_lgbm)}")
print("Submission file 'lgbm_updated_submission.csv' created successfully.")


In [ ]:
!pip install scikit-optimize

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from lightgbm import LGBMRegressor
from skopt import BayesSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# 데이터 로드
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# 데이터 확인
print("Train DataFrame columns:", train_df.columns)

In [ ]:
# 수치형과 범주형 특성 선택
numerical_features = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = train_df.select_dtypes(include=['object']).columns.tolist()

# 'ID'와 'y'는 제거
numerical_features = [col for col in numerical_features if col not in ['ID', 'y']]
categorical_features = [col for col in categorical_features if col != 'ID']

In [ ]:
print("Numerical features:", numerical_features)
print("Categorical features:", categorical_features)

# 결측값 대체 전략 설정
numerical_transformer = SimpleImputer(strategy='mean')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
# 컬럼 변환기를 통해 수치형과 범주형에 다른 전처리 적용
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'  # 나머지 컬럼은 그대로 유지
)

# 전처리 및 모델을 포함하는 파이프라인 구성
model = LGBMRegressor(random_state=42, objective='regression', early_stopping_rounds=10)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),  # 스케일링 추가
    ('model', model)
])

In [ ]:
# 하이퍼파라미터 검색 공간 정의
param_space_lgbm = {
    'model__n_estimators': (50, 2000),
    'model__max_depth': (1, 20),
    'model__learning_rate': (0.005, 0.3, 'log-uniform'),
    'model__subsample': (0.4, 1.0),
    'model__colsample_bytree': (0.4, 1.0),
    'model__min_child_samples': (1, 50),
    'model__reg_alpha': (0.0, 1.0),  # L1 regularization term
    'model__reg_lambda': (0.0, 1.0)  # L2 regularization term
}

# Bayesian Optimization 설정
opt_lgbm = BayesSearchCV(
    estimator=pipeline,
    search_spaces=param_space_lgbm,
    n_iter=100,
    cv=KFold(n_splits=5, shuffle=True, random_state=42),  # KFold 사용
    n_jobs=-1,
    random_state=42
)

In [ ]:
# 학습 데이터 준비
X_train = train_df.drop(columns=['ID', 'y'])
y_train = train_df['y']

# 결측값 확인 및 처리
print("Missing values in X_train before:", X_train.isnull().sum().sum())
print("Missing values in y_train before:", y_train.isnull().sum())

In [ ]:
# X_train의 결측값은 SimpleImputer로 처리되므로 따로 제거하지 않음
y_train = y_train.fillna(y_train.median())  # y_train의 결측값은 중앙값으로 대체

# 결측값 확인 후
print("Missing values in X_train after:", X_train.isnull().sum().sum())
print("Missing values in y_train after:", y_train.isnull().sum())

In [ ]:
# 모델 학습 및 최적의 하이퍼파라미터 찾기
opt_lgbm.fit(X_train, y_train)
print("Best hyperparameters for LightGBM:", opt_lgbm.best_params_)

# 특성 중요도 확인
best_model = opt_lgbm.best_estimator_.named_steps['model']
feature_importances = best_model.feature_importances_

In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor, Pool
from skopt import BayesSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 데이터 로드
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Feature Engineering
numerical_features = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = train_df.select_dtypes(include=['object']).columns.tolist()

numerical_features = [col for col in numerical_features if col not in ['ID', 'y']]
categorical_features = [col for col in categorical_features if col != 'ID']

# 데이터 타입 최적화
train_df[numerical_features] = train_df[numerical_features].astype(np.float32)
test_df[numerical_features] = test_df[numerical_features].astype(np.float32)

# 결측값 대체 전략 설정
numerical_transformer = SimpleImputer(strategy='mean')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features)
    ],
    remainder='passthrough'
)

# 학습 데이터 준비
X_train = train_df.drop(columns=['ID', 'y'])
y_train = train_df['y']

# 결측값 처리
y_train = y_train.fillna(y_train.median())

# 데이터 전처리
X_train_transformed = preprocessor.fit_transform(X_train)

# 파이프라인 설정
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('model', CatBoostRegressor(random_state=42, silent=True, task_type='GPU'))
])

# 하이퍼파라미터 검색 공간 정의
param_space = {
    'model__iterations': (100, 2000),
    'model__depth': (3, 15),
    'model__learning_rate': (0.01, 0.3, 'log-uniform'),
    'model__l2_leaf_reg': (1, 10),
    'model__bagging_temperature': (0.0, 1.0),
    'model__border_count': (32, 255)
}

# Bayesian Optimization 및 교차 검증 설정
cv = KFold(n_splits=5, shuffle=True, random_state=42)

opt = BayesSearchCV(
    estimator=pipeline,
    search_spaces=param_space,
    n_iter=50,
    cv=cv,  # KFold 교차 검증 사용
    n_jobs=1,  # 동시 작업자 수 제한
    random_state=42,
    scoring='neg_mean_squared_error'
)

# 하이퍼파라미터 최적화 수행
opt.fit(X_train, y_train, model__cat_features=categorical_features)
print("Best hyperparameters:")
print(opt.best_params_)

# 최적 하이퍼파라미터로 전체 학습 데이터로 모델 학습
best_model = opt.best_estimator_.named_steps['model']
train_pool = Pool(X_train_transformed, y_train, cat_features=categorical_features)

best_model.fit(
    train_pool,
    eval_set=train_pool,
    early_stopping_rounds=10,
    use_best_model=True,
    task_type='GPU'  # GPU 사용 설정
)

# 검증 데이터 예측
y_train_pred = best_model.predict(train_pool)
mse = mean_squared_error(y_train, y_train_pred)
r2 = r2_score(y_train, y_train_pred)
print(f"Training Mean Squared Error: {mse:.4f}")
print(f"Training R-squared: {r2:.4f}")

# 테스트 데이터 예측
X_test = test_df.drop(columns=['ID'])
X_test_transformed = preprocessor.transform(X_test)
test_pool = Pool(X_test_transformed, cat_features=categorical_features)

y_test_pred = best_model.predict(test_pool)

# 상위 33% 임계값 계산
threshold = np.percentile(y_test_pred, 67)
top_33_percent_mask = y_test_pred >= threshold

# 제출 파일 생성
submission_df = pd.read_csv('sample_submission.csv')
submission_df['y'] = y_test_pred
submission_df.to_csv('catboost_updated_submission.csv', index=False)

print(f"Top 33% threshold: {threshold:.4f}")
print(f"Number of samples in top 33%: {sum(top_33_percent_mask)}")
print("Submission file 'catboost_updated_submission.csv' created successfully.")


Training Mean Squared Error: 2.7204
Training R-squared: 0.6238
Top 33% threshold: 84.4735
Number of samples in top 33%: 1646
Submission file 'lgbm_updated_submission.csv' created successfully.

In [ ]:
!pip install lightgbm

In [ ]:
!pip install tabgan==1.3.3